In [16]:
# Deep Convolutional GANs

# Importing the libraries
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
import torchvision

# Setting some hyperparameters
batchSize = 64 # We set the size of the batch.
imageSize = 64 # We set the size of the generated images (64x64).

# Creating the transformations
transform = transforms.Compose([transforms.Resize(256),transforms.Scale(imageSize), transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),]) # We create a list of transformations (scaling, tensor conversion, normalization) to apply to the input images.

# Loading the dataset
#dataset = dset.CIFAR10(root = './data', download = True, transform = transform) # We download the training set in the ./data folder and we apply the previous transformations on each image.
#dataloader = torch.utils.data.DataLoader(dataset, batch_size = batchSize, shuffle = True, num_workers = 2) # We use dataLoader to get the images of the training set batch by batch.
#dataset = dset.ImageFolder(root="./images/", transform=transform)
#dataloader = torch.utils.data.DataLoader(dataset, batch_size = batchSize, shuffle = True, num_workers = 2)
dataset = dset.ImageFolder(root="./images",
                           transform=transforms.Compose([
                               transforms.Resize(imageSize),
                               transforms.CenterCrop(imageSize),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))
# Create the dataloader
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batchSize,
                                         shuffle=True, num_workers=2)

print("AAAAAA")


# Defining the weights_init function that takes as input a neural network m and that will initialize all its weights.
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

# Defining the generator


class G(nn.Module): # We introduce a class to define the generator.

    def __init__(self): # We introduce the __init__() function that will define the architecture of the generator.
        super(G, self).__init__() # We inherit from the nn.Module tools and activate inheritance
        self.main = nn.Sequential( # We create a meta module of a neural network that will contain a sequence of modules (convolutions, full connections, etc.).
            # inverse conv take input as random vector and return image, 100- size of input vector, 512- number of feature map of o/p, 4- size of kernel, 1- stride, 0-padding
            nn.ConvTranspose2d(100, 512, 4, 1, 0, bias = False), # We start with an inversed convolution. inverse cnn
            nn.BatchNorm2d(512), # We normalize all the features along the dimension of the batch.
            nn.ReLU(True), # We apply a ReLU rectification to break the linearity.
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias = False), # We add another inversed convolution.
            nn.BatchNorm2d(256), # We normalize again.
            nn.ReLU(True), # We apply another ReLU.
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias = False), # We add another inversed convolution.
            nn.BatchNorm2d(128), # We normalize again.
            nn.ReLU(True), # We apply another ReLU.
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias = False), # We add another inversed convolution.
            nn.BatchNorm2d(64), # We normalize again.
            nn.ReLU(True), # We apply another ReLU.
            nn.ConvTranspose2d(64, 3, 4, 2, 1, bias = False), # We add another inversed convolution.
            nn.Tanh() # We apply a Tanh rectification to break the linearity and stay between -1 and +1.
        )

    def forward(self, input): # We define the forward function that takes as argument an input that will be fed to the neural network, and that will return the output containing the generated images.
        output = self.main(input) # We forward propagate the signal through the whole neural network of the generator defined by self.main.
        return output # We return the output containing the generated images.


# Creating the generator
netG = G() # We create the generator object.
netG.apply(weights_init) # We initialize all the weights of its neural network.

# Defining the discriminator

class D(nn.Module): # We introduce a class to define the discriminator.

    def __init__(self): # We introduce the __init__() function that will define the architecture of the discriminator.
        super(D, self).__init__() # We inherit from the nn.Module tools.
        self.main = nn.Sequential( # We create a meta module of a neural network that will contain a sequence of modules (convolutions, full connections, etc.).
            # 3- channel of generated image generated by generated
            # 64-  feature map, 4- kernel size, 2- stride, 1- padding
            nn.Conv2d(3, 64, 4, 2, 1, bias = False), # We start with a convolution.
            nn.LeakyReLU(0.2, inplace = True), # We apply a LeakyReLU.
            nn.Conv2d(64, 128, 4, 2, 1, bias = False), # We add another convolution.
            nn.BatchNorm2d(128), # We normalize all the features along the dimension of the batch.
            nn.LeakyReLU(0.2, inplace = True), # We apply another LeakyReLU.
            nn.Conv2d(128, 256, 4, 2, 1, bias = False), # We add another convolution.
            nn.BatchNorm2d(256), # We normalize again.
            nn.LeakyReLU(0.2, inplace = True), # We apply another LeakyReLU.
            nn.Conv2d(256, 512, 4, 2, 1, bias = False), # We add another convolution.
            nn.BatchNorm2d(512), # We normalize again.
            nn.LeakyReLU(0.2, inplace = True), # We apply another LeakyReLU.
            nn.Conv2d(512, 1, 4, 1, 0, bias = False), # We add another convolution.
            # why sigmoid
            nn.Sigmoid() # We apply a Sigmoid rectification to break the linearity and stay between 0 and 1.
        )

    def forward(self, input): # We define the forward function that takes as argument an input that will be fed to the neural network, and that will return the output which will be a value between 0 and 1.
        output = self.main(input) # We forward propagate the signal through the whole neural network of the discriminator defined by self.main.
        return output.view(-1) # We return the output which will be a value between 0 and 1., view(-1) flaten the the results from the convolution layer

# Creating the discriminator
netD = D() # We create the discriminator object.
netD.apply(weights_init) # We initialize all the weights of its neural network.

# Training the DCGANs

criterion = nn.BCELoss() # We create a criterion object that will measure the error between the prediction and the target.
# lr- learning rate
optimizerD = optim.Adam(netD.parameters(), lr = 0.0002, betas = (0.5, 0.999)) # We create the optimizer object of the discriminator.
optimizerG = optim.Adam(netG.parameters(), lr = 0.0002, betas = (0.5, 0.999)) # We create the optimizer object of the generator.

for epoch in range(50): # We iterate over 25 epochs.
    # i = index, data- mini batch of the images
    for i, data in enumerate(dataloader, 0): # We iterate over the images of the dataset.
        
        # 1st Step: Updating the weights of the neural network of the discriminator

        netD.zero_grad() # We initialize to 0 the gradients of the discriminator with respect to the weights.
        
        # Training the discriminator with a real image of the dataset
        real, _ = data # We get a real image of the dataset which will be used to train the discriminator., 1st element
        input = Variable(real) # We wrap it in a variable. torch variable, minibach or images in a torch variable
        target = Variable(torch.ones(input.size()[0])) # We get the target.input.size()[0] is the size of the mini batch
        output = netD(input) # We forward propagate this real image into the neural network of the discriminator to get the prediction (a value between 0 and 1).
        errD_real = criterion(output, target) # We compute the loss between the predictions (output) and the target (equal to 1).
        
        # Training the discriminator with a fake image generated by the generator, size of vector size 100, as seen in 1st conv layer of generator
        noise = Variable(torch.randn(input.size()[0], 100, 1, 1)) # We make a random input vector (noise) of the generator.
        fake = netG(noise) # We forward propagate this random input vector into the neural network of the generator to get some fake generated images.
        target = Variable(torch.zeros(input.size()[0])) # We get the target.
        # detach means in this step we do not require gradient so we detach it
        # detach means in this step we do not require gradient so we detach it
        output = netD(fake.detach()) # We forward propagate the fake generated images into the neural network of the discriminator to get the prediction (a value between 0 and 1).
        errD_fake = criterion(output, target) # We compute the loss between the prediction (output) and the target (equal to 0).

        # Backpropagating the total error
        errD = errD_real + errD_fake # We compute the total error of the discriminator.
        errD.backward() # We backpropagate the loss error by computing the gradients of the total error with respect to the weights of the discriminator.
        optimizerD.step() # We apply the optimizer to update the weights according to how much they are responsible for the loss error of the discriminator.

        # 2nd Step: Updating the weights of the neural network of the generator

        netG.zero_grad() # We initialize to 0 the gradients of the generator with respect to the weights.
        target = Variable(torch.ones(input.size()[0])) # We get the target.
        output = netD(fake) # We forward propagate the fake generated images into the neural network of the discriminator to get the prediction (a value between 0 and 1).
        errG = criterion(output, target) # We compute the loss between the prediction (output between 0 and 1) and the target (equal to 1).
        errG.backward() # We backpropagate the loss error by computing the gradients of the total error with respect to the weights of the generator.
        optimizerG.step() # We apply the optimizer to update the weights according to how much they are responsible for the loss error of the generator.
        
        # 3rd Step: Printing the losses and saving the real images and the generated images of the minibatch every 100 steps
        print(errD.data.item())
        print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f' % (epoch, 25, i, len(dataloader), errD.data.item(), errG.data.item())) # We print les losses of the discriminator (Loss_D) and the generator (Loss_G).
        if i % 100 == 0: # Every 100 steps:
            vutils.save_image(real, '%s/real_samples.png' % "./results", normalize = True) # We save the real images of the minibatch.
            fake = netG(noise) # We get our fake generated images.
            vutils.save_image(fake.data, '%s/fake_samples_epoch_%03d.png' % ("./results", epoch), normalize = True) # We also save the fake generated images of the minibatch.

AAAAAA
1.7774043083190918
[0/25][0/16] Loss_D: 1.7774 Loss_G: 6.2913
0.9682199954986572
[0/25][1/16] Loss_D: 0.9682 Loss_G: 5.1473
0.9515328407287598
[0/25][2/16] Loss_D: 0.9515 Loss_G: 5.9763
0.7880936861038208
[0/25][3/16] Loss_D: 0.7881 Loss_G: 6.5571
0.8632767796516418
[0/25][4/16] Loss_D: 0.8633 Loss_G: 6.4296
0.7717856168746948
[0/25][5/16] Loss_D: 0.7718 Loss_G: 7.3537
1.132309913635254
[0/25][6/16] Loss_D: 1.1323 Loss_G: 6.8379
1.0952506065368652
[0/25][7/16] Loss_D: 1.0953 Loss_G: 9.4692
0.7333135604858398
[0/25][8/16] Loss_D: 0.7333 Loss_G: 7.8550
0.827343225479126
[0/25][9/16] Loss_D: 0.8273 Loss_G: 9.5525
0.6661596894264221
[0/25][10/16] Loss_D: 0.6662 Loss_G: 8.6829
0.9478237628936768
[0/25][11/16] Loss_D: 0.9478 Loss_G: 11.1878
0.6579517126083374
[0/25][12/16] Loss_D: 0.6580 Loss_G: 7.6384
1.7174357175827026
[0/25][13/16] Loss_D: 1.7174 Loss_G: 13.3064
0.5582208037376404
[0/25][14/16] Loss_D: 0.5582 Loss_G: 9.8988
1.1880247592926025
[0/25][15/16] Loss_D: 1.1880 Loss_G: 12

0.3475036919116974
[8/25][2/16] Loss_D: 0.3475 Loss_G: 2.5140
0.8824430108070374
[8/25][3/16] Loss_D: 0.8824 Loss_G: 4.1184
0.5842315554618835
[8/25][4/16] Loss_D: 0.5842 Loss_G: 3.9225
0.7845903635025024
[8/25][5/16] Loss_D: 0.7846 Loss_G: 3.1874
0.5166934132575989
[8/25][6/16] Loss_D: 0.5167 Loss_G: 3.9530
0.7191891670227051
[8/25][7/16] Loss_D: 0.7192 Loss_G: 3.4549
0.43526291847229004
[8/25][8/16] Loss_D: 0.4353 Loss_G: 3.6028
0.5281164646148682
[8/25][9/16] Loss_D: 0.5281 Loss_G: 4.3084
0.5581501722335815
[8/25][10/16] Loss_D: 0.5582 Loss_G: 2.5294
0.659511148929596
[8/25][11/16] Loss_D: 0.6595 Loss_G: 5.2526
0.5985652208328247
[8/25][12/16] Loss_D: 0.5986 Loss_G: 2.7244
0.539326548576355
[8/25][13/16] Loss_D: 0.5393 Loss_G: 3.1672
0.7543027400970459
[8/25][14/16] Loss_D: 0.7543 Loss_G: 6.6987
1.6776924133300781
[8/25][15/16] Loss_D: 1.6777 Loss_G: 2.0857
0.9891696572303772
[9/25][0/16] Loss_D: 0.9892 Loss_G: 4.6765
0.3572707772254944
[9/25][1/16] Loss_D: 0.3573 Loss_G: 4.6203
0.3

0.48114854097366333
[16/25][4/16] Loss_D: 0.4811 Loss_G: 6.5427
0.2673068046569824
[16/25][5/16] Loss_D: 0.2673 Loss_G: 5.4293
0.3950144350528717
[16/25][6/16] Loss_D: 0.3950 Loss_G: 7.3878
0.3917817771434784
[16/25][7/16] Loss_D: 0.3918 Loss_G: 5.0064
0.598250150680542
[16/25][8/16] Loss_D: 0.5983 Loss_G: 7.4784
0.29429835081100464
[16/25][9/16] Loss_D: 0.2943 Loss_G: 5.7244
0.4989706575870514
[16/25][10/16] Loss_D: 0.4990 Loss_G: 5.5096
0.46076667308807373
[16/25][11/16] Loss_D: 0.4608 Loss_G: 4.6022
0.8036748170852661
[16/25][12/16] Loss_D: 0.8037 Loss_G: 8.7196
1.6378660202026367
[16/25][13/16] Loss_D: 1.6379 Loss_G: 3.7874
0.9753974676132202
[16/25][14/16] Loss_D: 0.9754 Loss_G: 5.3684
0.35321423411369324
[16/25][15/16] Loss_D: 0.3532 Loss_G: 5.6954
0.819686233997345
[17/25][0/16] Loss_D: 0.8197 Loss_G: 4.7534
0.5033535957336426
[17/25][1/16] Loss_D: 0.5034 Loss_G: 6.1354
0.7021579742431641
[17/25][2/16] Loss_D: 0.7022 Loss_G: 6.0597
0.5891249179840088
[17/25][3/16] Loss_D: 0.5891

0.20669963955879211
[24/25][5/16] Loss_D: 0.2067 Loss_G: 4.8639
0.2696529030799866
[24/25][6/16] Loss_D: 0.2697 Loss_G: 6.4988
0.340154230594635
[24/25][7/16] Loss_D: 0.3402 Loss_G: 5.0915
0.3480527400970459
[24/25][8/16] Loss_D: 0.3481 Loss_G: 5.4434
0.5267242193222046
[24/25][9/16] Loss_D: 0.5267 Loss_G: 9.2399
0.8690812587738037
[24/25][10/16] Loss_D: 0.8691 Loss_G: 4.8714
0.8137472867965698
[24/25][11/16] Loss_D: 0.8137 Loss_G: 10.3250
0.3280889093875885
[24/25][12/16] Loss_D: 0.3281 Loss_G: 9.3104
0.21172365546226501
[24/25][13/16] Loss_D: 0.2117 Loss_G: 5.7681
0.8752583861351013
[24/25][14/16] Loss_D: 0.8753 Loss_G: 8.4239
0.4076743721961975
[24/25][15/16] Loss_D: 0.4077 Loss_G: 7.0231
0.3584781289100647
[25/25][0/16] Loss_D: 0.3585 Loss_G: 5.2841
0.5392645597457886
[25/25][1/16] Loss_D: 0.5393 Loss_G: 8.1527
0.3025188446044922
[25/25][2/16] Loss_D: 0.3025 Loss_G: 7.4619
0.4273218512535095
[25/25][3/16] Loss_D: 0.4273 Loss_G: 5.5610
1.026493787765503
[25/25][4/16] Loss_D: 1.0265 

0.3198264241218567
[32/25][6/16] Loss_D: 0.3198 Loss_G: 5.5680
0.23266056180000305
[32/25][7/16] Loss_D: 0.2327 Loss_G: 4.9792
0.18849603831768036
[32/25][8/16] Loss_D: 0.1885 Loss_G: 4.3688
0.27926602959632874
[32/25][9/16] Loss_D: 0.2793 Loss_G: 4.0320
0.26650553941726685
[32/25][10/16] Loss_D: 0.2665 Loss_G: 5.4302
0.1280277967453003
[32/25][11/16] Loss_D: 0.1280 Loss_G: 5.4791
0.20891684293746948
[32/25][12/16] Loss_D: 0.2089 Loss_G: 4.0920
0.22876349091529846
[32/25][13/16] Loss_D: 0.2288 Loss_G: 4.7187
0.22206412255764008
[32/25][14/16] Loss_D: 0.2221 Loss_G: 5.0260
0.25759071111679077
[32/25][15/16] Loss_D: 0.2576 Loss_G: 3.7600
0.3884340524673462
[33/25][0/16] Loss_D: 0.3884 Loss_G: 6.8945
0.36875149607658386
[33/25][1/16] Loss_D: 0.3688 Loss_G: 5.4881
0.0865863636136055
[33/25][2/16] Loss_D: 0.0866 Loss_G: 4.8139
0.15018807351589203
[33/25][3/16] Loss_D: 0.1502 Loss_G: 5.0015
0.17963896691799164
[33/25][4/16] Loss_D: 0.1796 Loss_G: 5.0010
0.18634626269340515
[33/25][5/16] Loss

0.18567807972431183
[40/25][7/16] Loss_D: 0.1857 Loss_G: 5.3212
0.3088928759098053
[40/25][8/16] Loss_D: 0.3089 Loss_G: 3.7635
0.3253611922264099
[40/25][9/16] Loss_D: 0.3254 Loss_G: 6.4992
0.3018002212047577
[40/25][10/16] Loss_D: 0.3018 Loss_G: 5.1411
0.32815659046173096
[40/25][11/16] Loss_D: 0.3282 Loss_G: 3.5852
0.3868691623210907
[40/25][12/16] Loss_D: 0.3869 Loss_G: 5.8053
0.16808289289474487
[40/25][13/16] Loss_D: 0.1681 Loss_G: 5.6266
0.1870923936367035
[40/25][14/16] Loss_D: 0.1871 Loss_G: 4.4785
0.17349253594875336
[40/25][15/16] Loss_D: 0.1735 Loss_G: 4.0588
0.4868246018886566
[41/25][0/16] Loss_D: 0.4868 Loss_G: 7.3395
0.4938415586948395
[41/25][1/16] Loss_D: 0.4938 Loss_G: 4.7519
0.23688867688179016
[41/25][2/16] Loss_D: 0.2369 Loss_G: 3.6604
0.2642250955104828
[41/25][3/16] Loss_D: 0.2642 Loss_G: 6.7467
0.12465605139732361
[41/25][4/16] Loss_D: 0.1247 Loss_G: 6.4967
0.3274981677532196
[41/25][5/16] Loss_D: 0.3275 Loss_G: 3.2339
0.36162373423576355
[41/25][6/16] Loss_D: 0

0.35357218980789185
[48/25][8/16] Loss_D: 0.3536 Loss_G: 4.5470
0.20158708095550537
[48/25][9/16] Loss_D: 0.2016 Loss_G: 4.4275
0.21645136177539825
[48/25][10/16] Loss_D: 0.2165 Loss_G: 3.9033
0.1641497015953064
[48/25][11/16] Loss_D: 0.1641 Loss_G: 5.3301
0.16688068211078644
[48/25][12/16] Loss_D: 0.1669 Loss_G: 4.5988
0.11321272701025009
[48/25][13/16] Loss_D: 0.1132 Loss_G: 4.7040
0.16213800013065338
[48/25][14/16] Loss_D: 0.1621 Loss_G: 5.1683
0.1033392921090126
[48/25][15/16] Loss_D: 0.1033 Loss_G: 5.3776
0.10740303993225098
[49/25][0/16] Loss_D: 0.1074 Loss_G: 4.8964
0.26843950152397156
[49/25][1/16] Loss_D: 0.2684 Loss_G: 3.0956
0.5665212273597717
[49/25][2/16] Loss_D: 0.5665 Loss_G: 8.9753
1.1511589288711548
[49/25][3/16] Loss_D: 1.1512 Loss_G: 4.0330
0.25007742643356323
[49/25][4/16] Loss_D: 0.2501 Loss_G: 4.0071
0.2486061006784439
[49/25][5/16] Loss_D: 0.2486 Loss_G: 6.3331
0.08079415559768677
[49/25][6/16] Loss_D: 0.0808 Loss_G: 6.4479
0.15789513289928436
[49/25][7/16] Loss_